In [12]:
recp_name = "s2"

In [13]:
YAML_BASE_PATH = f"/lustre/fs6/lyu_lab/scratch/ichen/data/boltz_runs/{recp_name}_rescore_cofold_config"
OUT_BASE_PATH = f"/lustre/fs6/lyu_lab/scratch/ichen/data/boltz_runs/{recp_name}_rescore_cofold_processed"

In [14]:
import os

In [15]:
yaml_files = os.listdir(YAML_BASE_PATH)

In [16]:
len(yaml_files)

506

In [17]:
array_num = 2

In [18]:
def split_list(lst, n):
    """
    Splits a list into n parts as evenly as possible.
    
    Args:
        lst (list): The input list.
        n (int): Number of parts.

    Returns:
        List of n lists.
    """
    k, r = divmod(len(lst), n)  # k = min size, r = remainder
    parts = []
    start = 0
    for i in range(n):
        end = start + k + (1 if i < r else 0)
        parts.append(lst[start:end])
        start = end
    return parts


In [19]:
parts = split_list(yaml_files, array_num)

In [20]:
def generate_job_name(selected_yaml_files):
    bash_array = "job_names=("
    for y in selected_yaml_files:
        bash_array += f"'{y}'\n "
    bash_array = bash_array.rstrip() + ")"

    return bash_array

In [21]:
counter = 0

for i in range(array_num):
    slurm_script = f"""#!/bin/bash
#SBATCH --job-name={i}_{recp_name}_rescore_cofold
#SBATCH --time=02:00:00
#SBATCH --partition=hpc_l40s
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --gres=gpu:4
#SBATCH --mem-per-gpu=40G
#SBATCH --output={recp_name}_%A.out

export CUDA_VISIBLE_DEVICES=0,1,2,3
export PATH=/ru-auth/local/home/ichen/miniconda3/bin:$PATH
source activate boltz_rescore

{generate_job_name(parts[i])}

# Build the tasks array
tasks=()
for JOB_NAME in "${{job_names[@]}}"; do
  tasks+=("boltz rescore {YAML_BASE_PATH}/${{JOB_NAME}} --out_dir {OUT_BASE_PATH}/$(basename $JOB_NAME .yaml) --use_msa_server")
done

printf "%s\\n" "${{tasks[@]}}" | parallel -j $SLURM_CPUS_PER_TASK
"""
    with open(f"{recp_name}_cofold_rescore_part{i}.sbatch", "w") as f:
        f.write(slurm_script)
